In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

### This part from  https://realpython.com/python-web-scraping-practical-introduction/

In [2]:
#https://realpython.com/python-web-scraping-practical-introduction/
def simple_get(url): 
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [3]:
quarters=['first','second','third','fourth']
webs=['https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-first-quarter-fiscal-year-2013',
'https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-second-quarter-fiscal-year-2013',
'https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-third-quarter-fiscal-year-2013',
'https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-annual-and-fourth-quarter-fiscal-2013',
'https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-first-quarter-fiscal-2014',
'https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-second-quarter-fiscal-2014',
'https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-third-quarter-fiscal-2014',
'https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-fourth-quarter-and-fiscal-2014',
'https://nvidianews.nvidia.com/news/nvidia-financial-results-for-first-quarter-fiscal-2015',
'https://nvidianews.nvidia.com/news/nvidia-financial-results-for-second-quarter-fiscal-2015',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-third-quarter-fiscal-2015',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-fourth-quarter-and-fiscal-2015',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-first-quarter-fiscal-2016',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-second-quarter-fiscal-2016',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-third-quarter-fiscal-2016',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-the-fourth-quarter-and-fiscal-2016',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-first-quarter-fiscal-2017',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-second-quarter-fiscal-2017',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-third-quarter-fiscal-2017',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-fourth-quarter-and-fiscal-2017',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-first-quarter-fiscal-2018',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-second-quarter-fiscal-2018',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-third-quarter-fiscal-2018',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-fourth-quarter-and-fiscal-2018',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-first-quarter-fiscal-2019',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-second-quarter-fiscal-2019',
'https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-third-quarter-fiscal-2019']
print(len(webs))

27


In [4]:
revenue=[]
grossmargin=[]
expenses=[]
netincome=[]
eps=[]
date = []
for web in webs:
    print(web)
    webg = simple_get(web)         
    webb = BeautifulSoup(webg, 'html.parser')
    
    ps = webb.find('p', {"class": "ct_title_sub"} )
    a = "".join(ps.text.strip().split(',')[1:])
    date.append(a.strip())     
    
    tables = webb.findAll('table')
    print(len(tables)) 
    data = []    
    table_body = tables[0].find('tbody')
    rows = table_body.find_all('tr')
 
    for row in rows:
        cells = row.find_all('td')
        texts = [td.text.strip() for td in cells if td != '']
        clean = [x for x in texts if str(x) != '$']
        clean = [x for x in clean if str(x) != '']
        data.append(clean) 
    
    df=pd.DataFrame(data=data)
    df[df.columns[0]] = df[df.columns[0]].str.lower() 
    df[df.columns[0]].replace("diluted earnings per share", value="earnings per share", inplace=True)
    df[df.columns[0]].replace("diluted earnings per share*", value="earnings per share", inplace=True)
    df[df.columns[0]].replace("net income*", value="net income", inplace=True)
    df = df.iloc[2:11,:]
    df.set_index(df.columns[0],inplace=True)

    revenue.append(df.loc['revenue',1])
    grossmargin.append(df.loc['gross margin',1])    
    expenses.append(df.loc['operating expenses',1])
    netincome.append(df.loc['net income',1])
    eps.append(df.loc['earnings per share',1])

https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-first-quarter-fiscal-year-2013
8
https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-second-quarter-fiscal-year-2013
7
https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-third-quarter-fiscal-year-2013
6
https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-annual-and-fourth-quarter-fiscal-2013
8
https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-first-quarter-fiscal-2014
6
https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-second-quarter-fiscal-2014
3
https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-third-quarter-fiscal-2014
6
https://nvidianews.nvidia.com/news/nvidia-reports-financial-results-for-fourth-quarter-and-fiscal-2014
6
https://nvidianews.nvidia.com/news/nvidia-financial-results-for-first-quarter-fiscal-2015
6
https://nvidianews.nvidia.com/news/nvidia-financial-results-for-second-qua

In [5]:
d={'Date':date,'Revenue':revenue,'Grossmargin':grossmargin,'Expenses':expenses,'Netincome':netincome,'EPS':eps}
d2f=pd.DataFrame(data=d)

d2f

,Date,EPS,Expenses,Grossmargin,Netincome,Revenue
0,May 11 2012,$0.10,$390.5,50.1%,$60.4,$924.9
1,August 9 2012,$0.19,$401.1,51.8%,$119.0,"$1,044.3"
2,November 8 2012,$0.33,$384.4,52.9%,$209.1,"$1,204.1"
3,February 13 2013,$0.90,"$1,578.1",52.0%,$562.5,"$4,280.2"
4,May 9 2013,$0.13,$435.8,54.3%,$77.9,$954.7
5,August 8 2013,$0.16,$440.0,55.8%,$96.4,$977.2
6,November 7 2013,$0.20,$443.4,55.4%,$118.7,"$1,054.0"
7,February 12 2014,$0.25,$452,54.1%,$147,"$1,144"
8,May 8 2014,$0.24,$452.8,54.8%,$136.5,"$1,102.8"
9,August 7 2014,0.22,456,56.1%,128,"1,103"


In [6]:
for x in d2f.columns:
    d2f[x] = d2f[x].str.replace(',', '')
    d2f[x] = d2f[x].str.replace('$', '') 
    d2f[x] = d2f[x].str.replace('%', '') 
d2f

,Date,EPS,Expenses,Grossmargin,Netincome,Revenue
0,May 11 2012,0.10,390.5,50.1,60.4,924.9
1,August 9 2012,0.19,401.1,51.8,119.0,1044.3
2,November 8 2012,0.33,384.4,52.9,209.1,1204.1
3,February 13 2013,0.90,1578.1,52.0,562.5,4280.2
4,May 9 2013,0.13,435.8,54.3,77.9,954.7
5,August 8 2013,0.16,440.0,55.8,96.4,977.2
6,November 7 2013,0.20,443.4,55.4,118.7,1054.0
7,February 12 2014,0.25,452,54.1,147,1144
8,May 8 2014,0.24,452.8,54.8,136.5,1102.8
9,August 7 2014,0.22,456,56.1,128,1103


In [7]:
d2f.Date = pd.to_datetime(d2f.Date)
d2f

,Date,EPS,Expenses,Grossmargin,Netincome,Revenue
0,2012-05-11,0.10,390.5,50.1,60.4,924.9
1,2012-08-09,0.19,401.1,51.8,119.0,1044.3
2,2012-11-08,0.33,384.4,52.9,209.1,1204.1
3,2013-02-13,0.90,1578.1,52.0,562.5,4280.2
4,2013-05-09,0.13,435.8,54.3,77.9,954.7
5,2013-08-08,0.16,440.0,55.8,96.4,977.2
6,2013-11-07,0.20,443.4,55.4,118.7,1054.0
7,2014-02-12,0.25,452,54.1,147,1144
8,2014-05-08,0.24,452.8,54.8,136.5,1102.8
9,2014-08-07,0.22,456,56.1,128,1103


In [8]:
from urllib.request import urlopen 
from io import BytesIO
from tabula import read_pdf
url = 'https://s22.q4cdn.com/364334381/files/doc_financials/quarterly_reports/2016/Rev_by_Mkt_Qtrly_Trend.pdf'
pdf = BytesIO(urlopen(url).read() )
df1 = read_pdf(pdf, spreadsheet=True, area=(37, 26, 492, 884))
df1.rename(columns = {'($ in millions)':'Date'},inplace=True)
df1['Date'].replace("Pro Visualization", value="ProVision", inplace=True)
df1[df1.columns[0]].replace("PC & Tegra OEM / IP", value="OEM", inplace=True)

df1 

,Date,Q1 FY15,Q2 FY15,Q3 FY15,Q4 FY15,Q1 FY16,Q2 FY16,Q3 FY16
0,Gaming,$ 468,$ 417,$ 528,$ 646,$ 587,$ 661,$ 761
1,ProVision,191,208,206,190,181,176,190
2,Datacenter,57,83,90,88,88,73,82
3,Auto,35,40,52,56,77,71,79
4,OEM,352,355,350,271,219,173,192
5,Total,"$ 1,103","$ 1,103","$ 1,225","$ 1,251","$ 1,151","$ 1,153","$ 1,305"


In [9]:
url = 'https://s22.q4cdn.com/364334381/files/doc_financials/quarterly_reports/2018/Rev_by_Mkt_Qtrly_Trend_Q318.pdf'
pdf = BytesIO(urlopen(url).read() )
df2 = read_pdf(pdf, spreadsheet=True, area=(37, 26, 492, 884))
df2.rename(columns = {'($ in millions)':'Date'},inplace=True)
df2['Date'].replace("Professional\rVisualization", value="ProVision", inplace=True)
df2[df2.columns[0]].replace("OEM & IP", value="OEM", inplace=True)
df2

,Date,Q4 FY16,Q1 FY17,Q2 FY17,Q3 FY17,Q4 FY17,Q1 FY18,Q2 FY18,Q3 FY18
0,Gaming,$ 810,$ 687,$ 781,"$ 1,244","$ 1,348","$ 1,027","$ 1,186","$ 1,561"
1,ProVision,203,189,214,207,225,205,235,239
2,Datacenter,97,143,151,240,296,409,416,501
3,Auto,93,113,119,127,128,140,142,144
4,OEM,198,173,163,186,176,156,251,191
5,Total,"$ 1,401","$ 1,305","$ 1,428","$ 2,004","$ 2,173","$ 1,937","$ 2,230","$ 2,636"


In [10]:
url = 'https://s22.q4cdn.com/364334381/files/doc_financials/quarterly_reports/2019/Q319/Rev_by_Mkt_Qtrly_Trend_Q319.pdf'
pdf = BytesIO(urlopen(url).read() )
df3 = read_pdf(pdf, spreadsheet=True, area=(37, 26, 492, 884))
df3.drop(['Q3 FY18','Q2 FY18','Q1 FY18','Q4 FY17'],axis=1,inplace=True)  
df3.rename(columns = {'($ in millions)':'Date'},inplace=True)
df3['Date'].replace("Professional\rVisualization", value="ProVision", inplace=True)
df3['Date'].replace("OEM & IP", value="OEM", inplace=True)
df3 = df3[df3.columns[::-1]]
df3

,Q4 FY18,Q1 FY19,Q2 FY19,Q3 FY19,Date
0,"$ 1,739","$ 1,723","$ 1,805","$ 1,764",Gaming
1,254,251,281,305,ProVision
2,606,701,760,792,Datacenter
3,132,145,161,172,Auto
4,180,387,116,148,OEM
5,"$ 2,911","$ 3,207","$ 3,123","$ 3,181",TOTAL


In [11]:
df4 = pd.merge(df1, df2, on='Date')
df4 = pd.merge(df4, df3, on='Date')
df4

,Date,Q1 FY15,Q2 FY15,Q3 FY15,Q4 FY15,Q1 FY16,Q2 FY16,Q3 FY16,Q4 FY16,Q1 FY17,Q2 FY17,Q3 FY17,Q4 FY17,Q1 FY18,Q2 FY18,Q3 FY18,Q4 FY18,Q1 FY19,Q2 FY19,Q3 FY19
0,Gaming,$ 468,$ 417,$ 528,$ 646,$ 587,$ 661,$ 761,$ 810,$ 687,$ 781,"$ 1,244","$ 1,348","$ 1,027","$ 1,186","$ 1,561","$ 1,739","$ 1,723","$ 1,805","$ 1,764"
1,ProVision,191,208,206,190,181,176,190,203,189,214,207,225,205,235,239,254,251,281,305
2,Datacenter,57,83,90,88,88,73,82,97,143,151,240,296,409,416,501,606,701,760,792
3,Auto,35,40,52,56,77,71,79,93,113,119,127,128,140,142,144,132,145,161,172
4,OEM,352,355,350,271,219,173,192,198,173,163,186,176,156,251,191,180,387,116,148


In [12]:
for x in df4.columns:
    df4[x] = df4[x].str.replace(',', '')
    df4[x] = df4[x].str.replace('$', '') 
print(df4.shape)
df4

(5, 20)


,Date,Q1 FY15,Q2 FY15,Q3 FY15,Q4 FY15,Q1 FY16,Q2 FY16,Q3 FY16,Q4 FY16,Q1 FY17,Q2 FY17,Q3 FY17,Q4 FY17,Q1 FY18,Q2 FY18,Q3 FY18,Q4 FY18,Q1 FY19,Q2 FY19,Q3 FY19
0,Gaming,468,417,528,646,587,661,761,810,687,781,1244,1348,1027,1186,1561,1739,1723,1805,1764
1,ProVision,191,208,206,190,181,176,190,203,189,214,207,225,205,235,239,254,251,281,305
2,Datacenter,57,83,90,88,88,73,82,97,143,151,240,296,409,416,501,606,701,760,792
3,Auto,35,40,52,56,77,71,79,93,113,119,127,128,140,142,144,132,145,161,172
4,OEM,352,355,350,271,219,173,192,198,173,163,186,176,156,251,191,180,387,116,148


In [13]:
df4.columns = d2f.Date[7:]
df5 = df4.T
df5.rename(columns=df5.iloc[0],inplace=True)
df5 = df5.iloc[1:]
df5

,Gaming,ProVision,Datacenter,Auto,OEM
Date,,,,,
2014-05-08,468,191,57,35,352
2014-08-07,417,208,83,40,355
2014-11-06,528,206,90,52,350
2015-02-11,646,190,88,56,271
2015-05-07,587,181,88,77,219
2015-08-06,661,176,73,71,173
2015-11-05,761,190,82,79,192
2016-02-17,810,203,97,93,198
2016-05-12,687,189,143,113,173


In [14]:
print(df5.shape)
print(d2f[8:].shape)
df6 = pd.merge(d2f[8:], df5, on='Date')
df6

(19, 5)
(19, 6)


,Date,EPS,Expenses,Grossmargin,Netincome,Revenue,Gaming,ProVision,Datacenter,Auto,OEM
0,2014-05-08,0.24,452.8,54.8,136.5,1102.8,468,191,57,35,352
1,2014-08-07,0.22,456,56.1,128,1103,417,208,83,40,355
2,2014-11-06,0.31,463,55.2,173,1225,528,206,90,52,350
3,2015-02-11,0.35,468,55.9,193,1251,646,190,88,56,271
4,2015-05-07,0.24,477,56.7,134,1151,587,181,88,77,219
5,2015-08-06,0.05,558,55.0,26,1153,661,176,73,71,173
6,2015-11-05,0.44,489,56.3,246,1305,761,190,82,79,192
7,2016-02-17,0.35,539,56.5,207,1401,810,203,97,93,198
8,2016-05-12,0.33,506,57.5,196,1305,687,189,143,113,173
9,2016-08-11,0.40,509,57.9,253,1428,781,214,151,119,163


In [15]:
webg = simple_get('https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end=20190102')         
webb = BeautifulSoup(webg, 'html.parser')
  
tables = webb.findAll('table')
print(len(tables)) 
data = []    
table_body = tables[0].find('tbody')
rows = table_body.find_all('tr')
 
for row in rows:
    cells = row.find_all('td')
    texts = [td.text.strip() for td in cells if td != '']
    clean = [x for x in texts if str(x) != '$']
    clean = [x for x in clean if str(x) != '']
    data.append(clean) 
    
df8=pd.DataFrame(data=data)

1


In [16]:
df8.columns=['Date','Open','High','Low','Close','Volume','MarketCap']
df8.Date = pd.to_datetime(df8.Date)

for x in df8.columns[1:]:
    df8[x] = df8[x].astype(str)  
    df8[x] = df8[x].str.replace(',', '')
    df8[x] = df8[x].str.replace('$', '') 
    df8[x] = df8[x].str.replace('%', '')
    df8[x] = df8[x].str.replace('-', '0')
    df8[x] = df8[x].astype(float)

print(df8.shape)

df8.head()

(2076, 7)


,Date,Open,High,Low,Close,Volume,MarketCap
0,2019-01-02,3849.22,3947.98,3817.41,3943.41,5.244857e+09,6.884986e+10
1,2019-01-01,3746.71,3850.91,3707.23,3843.52,4.324201e+09,6.709863e+10
2,2018-12-31,3866.84,3868.74,3725.87,3742.70,4.661841e+09,6.533150e+10
3,2018-12-30,3822.38,3901.91,3797.22,3865.95,4.770579e+09,6.747551e+10
4,2018-12-29,3932.49,3963.76,3820.41,3820.41,4.991656e+09,6.667224e+10


In [17]:
df9 = df8[df8.index.isin(df6.index)]
df9 = df8[df8.Date.isin(df6.Date)]
df9.set_index('Date',inplace=True)
df10 = df9[['Close']]
df10.columns = ['Bitcoin']
df10

,Bitcoin
Date,
2018-11-15,5648.03
2018-08-16,6334.73
2018-05-10,9043.94
2018-02-08,8265.59
2017-11-09,7143.58
2017-08-10,3381.28
2017-05-09,1755.36
2017-02-09,994.38
2016-11-10,715.53


In [18]:
df11 = pd.merge(df6, df10, on='Date')
df11

,Date,EPS,Expenses,Grossmargin,Netincome,Revenue,Gaming,ProVision,Datacenter,Auto,OEM,Bitcoin
0,2014-05-08,0.24,452.8,54.8,136.5,1102.8,468,191,57,35,352,440.17
1,2014-08-07,0.22,456,56.1,128,1103,417,208,83,40,355,588.87
2,2014-11-06,0.31,463,55.2,173,1225,528,206,90,52,350,349.29
3,2015-02-11,0.35,468,55.9,193,1251,646,190,88,56,271,219.18
4,2015-05-07,0.24,477,56.7,134,1151,587,181,88,77,219,237.33
5,2015-08-06,0.05,558,55.0,26,1153,661,176,73,71,173,278.58
6,2015-11-05,0.44,489,56.3,246,1305,761,190,82,79,192,386.35
7,2016-02-17,0.35,539,56.5,207,1401,810,203,97,93,198,416.32
8,2016-05-12,0.33,506,57.5,196,1305,687,189,143,113,173,454.77
9,2016-08-11,0.40,509,57.9,253,1428,781,214,151,119,163,589.12
